In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch

In [2]:
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'

In [3]:
data = pd.read_csv('../input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict,
                       index_col = 0, chunksize = 3141410) #3141410

In [4]:
data = next(iter(data))

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
#plt.plot(data.groupby('time_id').count())
#plt.plot()

In [6]:
"""
time_id = 40 #40
X_train = data.loc[:time_id-1,]
X_test = data.loc[time_id:,]
print(X_train.shape[0], X_test.shape[0])

y_train = X_train['target']
X_train.drop(['target', 'investment_id'], inplace=True, axis=1)

y_test = X_test['target']
X_test.drop(['target', 'investment_id'], inplace=True, axis=1)
print(y_train.shape[0], y_test.shape[0])
print(y_test.shape[0]/y_train.shape[0])
"""

"\ntime_id = 40 #40\nX_train = data.loc[:time_id-1,]\nX_test = data.loc[time_id:,]\nprint(X_train.shape[0], X_test.shape[0])\n\ny_train = X_train['target']\nX_train.drop(['target', 'investment_id'], inplace=True, axis=1)\n\ny_test = X_test['target']\nX_test.drop(['target', 'investment_id'], inplace=True, axis=1)\nprint(y_train.shape[0], y_test.shape[0])\nprint(y_test.shape[0]/y_train.shape[0])\n"

In [7]:
target = data['target']
data.drop(['target', 'investment_id'], inplace=True, axis=1)

In [8]:
from scipy.stats import pearsonr 
def est_score(model):
    y_pred = model.predict(X_test)
    return pearsonr(y_pred, y_test)[0]

In [9]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, data['target'], test_size=0.1, stratify=data.index)

#id_train = X_train['investment_id']
#id_test = X_test['investment_id']

#X_train.drop(['target', 'investment_id'], inplace=True, axis=1)
#X_test.drop(['target', 'investment_id'], inplace=True, axis=1)

In [10]:
#id_train.isna().sum()

In [11]:
#del data

In [12]:
#from sklearn.linear_model import LinearRegression
#reg_model = LinearRegression(copy_X=False).fit(X_train, y_train)

In [13]:
#est_score(reg_model)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_to(data, device):
    """
    moving data to device
    :param data: data to move
    :param device: device
    :return: moved data
    """

    if isinstance(data, (list, tuple)):
        return [move_to(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [15]:
def batch_loader(X, y, batch_size = 4096, equal=False):
    i = 0
    while True:
        if i + batch_size >= y.shape[0]: 
            break
            
        yield X.iloc[i:i+batch_size, :], y[i:i+batch_size]
        i += batch_size

In [16]:
def pearson_loss(x, y):
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    loss = -torch.sum(vx*vy) / (torch.sqrt(torch.sum(vx*vx)) * torch.sqrt(torch.sum(vy*vy)))
    return loss

In [17]:
class DNN_Model(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 0
        self.model_name = 'DNN_Model_0'
        self.linear_1 = torch.nn.Linear(300, deep_size)
        self.linear_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear_3 = torch.nn.Linear(deep_size, deep_size)
        self.linear_4 = torch.nn.Linear(deep_size, deep_size)
        self.linear_5 = torch.nn.Linear(deep_size, 1)
        self.dout = torch.nn.Dropout(p=0.7)
        self.act = torch.nn.SiLU()
    
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        tensor_X = self.act(self.linear_1(X_tensor))
        tensor_X = self.act(self.linear_2(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_3(tensor_X))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear_4(tensor_X))
        tensor_X = self.linear_5(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [18]:
class DNN_Model_1(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 1
        self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, 1)
        )
        
        self.model_name = 'DNN_Model_1'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [19]:
class DNN_Model_2(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 2
        act_1 = torch.nn.SiLU
        act_2 = torch.nn.ReLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act_1(),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, 1)
            
        )
        
        self.model_name = 'DNN_Model_2'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [20]:
class DNN_Model_3(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 3
        self.linear1 = torch.nn.Linear(300, deep_size)
        
        self.linear2_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear3_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear4_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear5_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear6_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear6_2 = torch.nn.Linear(deep_size, 1)
        
        self.act = torch.nn.SiLU()
        self.dout = torch.nn.Dropout(p=0.5)
        
        self.model_name = 'DNN_Model_3'

                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        
        X_tensor = self.act(self.linear1(X_tensor))
        
        X_tensor = self.act(self.linear2_1(X_tensor))
        X_tensor = self.act(self.linear2_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_2 = self.act(self.linear2_3(X_tensor))
        
        
        #res_2 = X_tensor
        
        X_tensor = self.act(self.linear3_1(res_2))
        X_tensor = self.act(self.linear3_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_3 = self.act(self.linear3_3(X_tensor))
        
        
        #res_3 = X_tensor
        
        X_tensor = self.act(self.linear4_1(res_2 + res_3))
        X_tensor = self.act(self.linear4_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_4 = self.act(self.linear4_3(X_tensor))
        
        res_4 = X_tensor

        X_tensor = self.act(self.linear5_1(res_3 + res_4))
        X_tensor = self.act(self.linear5_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        X_tensor = self.act(self.linear5_3(X_tensor))
        
        X_tensor = self.act(self.linear6_1(X_tensor + res_4))
        X_tensor = self.linear6_2(X_tensor)
        
        return X_tensor
        

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [21]:
class DNN_Model_4(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 4
        act_1 = torch.nn.SiLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, 2*deep_size),
            
            torch.nn.Dropout(p=0.4),
            
            torch.nn.Linear(2*deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, 2*deep_size),
            act_1(),
            torch.nn.Linear(2*deep_size, deep_size),
            act_1(),
            
            torch.nn.Dropout(p=0.4),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, 1)
            
        )
        
        self.model_name = 'DNN_Model_4'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [22]:
class DNN_Model_5(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 5
        act = torch.nn.SiLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act(),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.Linear(deep_size, 1)
            
        )
        
        self.model_name = 'DNN_Model_5'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [23]:
class DNN_Model_6(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.deep_size = deep_size
        self.index = 6
        act = torch.nn.SiLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act(),
            torch.nn.Linear(deep_size, 256),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(256, 256),
            act(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(256, 384),
            act(),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(384, 384),
            act(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(384, 256),
            act(),
            torch.nn.Linear(256, deep_size),
            act(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(deep_size, deep_size),
            act(),
            torch.nn.Linear(deep_size, 64),
            act(),
            torch.nn.Linear(64, 1)
        )
        self.model_name = 'DNN_Model_6'
        
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [24]:
dnn_models = [DNN_Model, DNN_Model_1, DNN_Model_2, DNN_Model_3, DNN_Model_4, DNN_Model_5]

In [25]:
#написать subsample еще раз (также попробовать booststrap)
#написать еще пару моделей, желательно как можно более разных
#из большого количества моделей выбрать топ лучших (на полном датасете!)
#продумать как можно (например с помощью матрицы корреляции) стакать сети в одной модели чтобы увеличить скор
#поработать с бустами, попробовать их пообъединять

In [26]:
def corr_based_coefs(a):
    if a.shape[0] < 2:
        return np.array([1])
    preds_sum = np.sum(a, axis=0)
    corrs = np.array([pearsonr( (preds_sum-a[i])/(a.shape[0]-1), a[i])[0] for i in range(a.shape[0])])
    print(f'Correlation coefs: {corrs}')
    corrs = 1 - corrs
    corrs /= np.sum(corrs)
    print(f'Stacking coefs: {corrs}')
    return corrs

In [27]:
class StackedDNN:
    def __init__(self, n, model, min_deep=30, max_deep=40):
        self.nets = [model(np.random.randint(min_deep, max_deep)) for i in range(n)]
        self.model_name = self.nets[0].model_name
        for net in self.nets:
            net = move_to(net, device)
    
    def fit(self, X, y, iters, batch_size=4096, subset=1, float_subset=True, silent=False, logs=False, plot=False, val=False, X_val=None, y_val=None):
        
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
            
        if logs and silent:
            print(f'Net {self.nets[0].model_name}')
            
        for i in range(len(self.nets)):
    
            scores_train = []
            scores_val = []
        
            if val:
                tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
                tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
            scores_train = []
            scores_val = []

            opt = torch.optim.Adam(self.nets[i].parameters(), lr=5e-4)

            if float_subset:
                subset_ind = np.random.uniform(subset, 1)

            subset_ind = np.random.choice(batch_size, int(batch_size*subset))
            
            if not silent:
                print(f'>>> Net {i + 1}/{len(self.nets)}')
                print(f'Deep size: {self.nets[i].deep_size}')
                print()
                
                
            for epoch in range(iters):
                offset = 0
                #learning cycle
                while offset + batch_size < y.shape[0]: 
                    self.nets[i].train()
                    self.nets[i].zero_grad()
                    X_tensor = move_to(torch.FloatTensor(X.iloc[offset:offset+batch_size, :].values), device)
                    y_tensor = move_to(torch.FloatTensor(y[offset:offset+batch_size].values), device)
                    X_tensor = X_tensor[subset_ind,:]
                    y_tensor = y_tensor[subset_ind]
                    
                    loss = pearson_loss(self.nets[i].tensor_forward(X_tensor).view(-1), y_tensor)
                    loss.backward()
                    opt.step()
                    offset += batch_size

                #prints
                if val:
                    self.nets[i].eval()
                    val_loss = pearson_loss(self.nets[i].tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                    scores_val.append(float(val_loss.detach()))
                scores_train.append(float(loss.detach()))

                if not silent:
                    print(f'epoch {epoch} finished with {loss} loss')
                    if val:
                        print(f'val loss {val_loss}')
                
            if logs and silent:
                print(f'Net {i+1}/{len(self.nets)} fitted')
                
            if not silent:
                if val:
                    print(f'Finished with \n train loss: {loss}\n val loss: {val_loss}')
                else:
                    print(f'Finished with {loss} loss')
            if plot:
                if val:
                    sns.lineplot(x=range(iters), y=scores_train, label='train')
                    sns.lineplot(x=range(iters), y=scores_val, label='val')
                else:
                    sns.lineplot(scores_train)
                plt.plot()

        
    def predict(self, X):
        y_pred = np.ndarray([len(self.nets), X.shape[0]])
        for i in range(len(self.nets)):
            self.nets[i].eval()
            y_pred[i] = np.array(self.nets[i].forward(X).detach().view(-1).cpu())
        y_pred = corr_based_coefs(y_pred).dot(y_pred)
        return y_pred
    
    def save(self):
        import os
        try:
            os.mkdir(f'./{self.model_name}')
        except:
            pass
        with open(f'./{self.model_name}/config.txt', 'w') as f:
            f.write(str(self.nets[0].index) + '\n')
            for i in range(len(self.nets)):
                torch.save(self.nets[i].state_dict(), f'./{self.model_name}/model:{i}')
                f.write(str(self.nets[i].deep_size) + '\n')

    
                       
    def load(self):
        with open(f'../input/ubiquantdevelop/{self.model_name}/config.txt', 'r') as f:
            dnn_model = dnn_models[int(f.readline())]
            
            for i in range(len(self.nets)):
                self.nets[i] = dnn_model(int(f.readline()))
                self.nets[i].load_state_dict(torch.load(f'../input/ubiquantdevelop/{self.model_name}/model:{i}', map_location=torch.device('cpu')))
                self.nets[i] = move_to(self.nets[i], device)

In [28]:
#0   0.16049979767507658
#1   0.16195405253013023
#2   0.16213173833206326
#3   0.16256979869460275
#4

#написать ноутбук как меняется необходимое количество эпох в завимисимости от размера выбранной части выборки
#то есть для подвыборки лучшее качетство получается на n итерациях, как дело обстоит если увеличивать выборку
#лучше взять датасет для регресии и поменьше
#можно взять несколько датасетов

In [29]:
nn_model = StackedDNN(10, DNN_Model, min_deep=280, max_deep=301)
nn_model.fit(data, target, 5, subset=1)
nn_model.save()

>>> Net 1/10
Deep size: 288

epoch 0 finished with -0.24693968892097473 loss
epoch 1 finished with -0.24460723996162415 loss
epoch 2 finished with -0.2522820830345154 loss
epoch 3 finished with -0.2472161054611206 loss
epoch 4 finished with -0.2496943175792694 loss
Finished with -0.2496943175792694 loss
>>> Net 2/10
Deep size: 294

epoch 0 finished with -0.24021734297275543 loss
epoch 1 finished with -0.2567983567714691 loss
epoch 2 finished with -0.2594028413295746 loss
epoch 3 finished with -0.2606032192707062 loss
epoch 4 finished with -0.2746770977973938 loss
Finished with -0.2746770977973938 loss
>>> Net 3/10
Deep size: 288

epoch 0 finished with -0.23524922132492065 loss
epoch 1 finished with -0.23356597125530243 loss
epoch 2 finished with -0.24730341136455536 loss
epoch 3 finished with -0.23560263216495514 loss
epoch 4 finished with -0.2474345564842224 loss
Finished with -0.2474345564842224 loss
>>> Net 4/10
Deep size: 282

epoch 0 finished with -0.23979631066322327 loss
epoch 1

In [30]:
nn_model_1 = StackedDNN(10, DNN_Model_1, min_deep=40, max_deep=41)
nn_model_1.fit(data, target, 5, subset=1)
nn_model_1.save()

>>> Net 1/10
Deep size: 40

epoch 0 finished with -0.1880437135696411 loss
epoch 1 finished with -0.1933516263961792 loss
epoch 2 finished with -0.187080979347229 loss
epoch 3 finished with -0.20494689047336578 loss
epoch 4 finished with -0.16735270619392395 loss
Finished with -0.16735270619392395 loss
>>> Net 2/10
Deep size: 40

epoch 0 finished with -0.17569400370121002 loss
epoch 1 finished with -0.16813808679580688 loss
epoch 2 finished with -0.1794738918542862 loss
epoch 3 finished with -0.17088033258914948 loss
epoch 4 finished with -0.178660586476326 loss
Finished with -0.178660586476326 loss
>>> Net 3/10
Deep size: 40

epoch 0 finished with -0.20426365733146667 loss
epoch 1 finished with -0.20232141017913818 loss
epoch 2 finished with -0.20594707131385803 loss
epoch 3 finished with -0.21068470180034637 loss
epoch 4 finished with -0.19789838790893555 loss
Finished with -0.19789838790893555 loss
>>> Net 4/10
Deep size: 40

epoch 0 finished with -0.25511592626571655 loss
epoch 1 f

In [31]:
nn_model_2 = StackedDNN(10, DNN_Model_2, min_deep=100, max_deep=101)
nn_model_2.fit(data, target, 5, subset=1)
nn_model_2.save()

>>> Net 1/10
Deep size: 100

epoch 0 finished with -0.2972073554992676 loss
epoch 1 finished with -0.3069702088832855 loss
epoch 2 finished with -0.3098713457584381 loss
epoch 3 finished with -0.2991941273212433 loss
epoch 4 finished with -0.3147832453250885 loss
Finished with -0.3147832453250885 loss
>>> Net 2/10
Deep size: 100

epoch 0 finished with -0.2667606472969055 loss
epoch 1 finished with -0.26880964636802673 loss
epoch 2 finished with -0.27846023440361023 loss
epoch 3 finished with -0.2770363986492157 loss
epoch 4 finished with -0.2706456184387207 loss
Finished with -0.2706456184387207 loss
>>> Net 3/10
Deep size: 100

epoch 0 finished with -0.27953994274139404 loss
epoch 1 finished with -0.2714928984642029 loss
epoch 2 finished with -0.2649078369140625 loss
epoch 3 finished with -0.2595667243003845 loss
epoch 4 finished with -0.25946491956710815 loss
Finished with -0.25946491956710815 loss
>>> Net 4/10
Deep size: 100

epoch 0 finished with -0.2936372458934784 loss
epoch 1 fi

In [32]:
nn_model_3 = StackedDNN(10, DNN_Model_3, min_deep=90, max_deep=110)
nn_model_3.fit(data, target, 5, subset=1)
nn_model_3.save()

>>> Net 1/10
Deep size: 97

epoch 0 finished with -0.24891239404678345 loss
epoch 1 finished with -0.2649911344051361 loss
epoch 2 finished with -0.26951995491981506 loss
epoch 3 finished with -0.2752933204174042 loss
epoch 4 finished with -0.28771039843559265 loss
Finished with -0.28771039843559265 loss
>>> Net 2/10
Deep size: 100

epoch 0 finished with -0.199471116065979 loss
epoch 1 finished with -0.19169989228248596 loss
epoch 2 finished with -0.18833041191101074 loss
epoch 3 finished with -0.1975925713777542 loss
epoch 4 finished with -0.19214491546154022 loss
Finished with -0.19214491546154022 loss
>>> Net 3/10
Deep size: 92

epoch 0 finished with -0.21249793469905853 loss
epoch 1 finished with -0.20998099446296692 loss
epoch 2 finished with -0.2125391811132431 loss
epoch 3 finished with -0.22337943315505981 loss
epoch 4 finished with -0.22080321609973907 loss
Finished with -0.22080321609973907 loss
>>> Net 4/10
Deep size: 97

epoch 0 finished with -0.18578170239925385 loss
epoch

In [33]:
nn_model_4 = StackedDNN(10, DNN_Model_4, min_deep=90, max_deep=110)
nn_model_4.fit(data, target, 5, subset=1)
nn_model_4.save()

>>> Net 1/10
Deep size: 92

epoch 0 finished with -0.2529107630252838 loss
epoch 1 finished with -0.228806272149086 loss
epoch 2 finished with -0.23439006507396698 loss
epoch 3 finished with -0.24320636689662933 loss
epoch 4 finished with -0.23174431920051575 loss
Finished with -0.23174431920051575 loss
>>> Net 2/10
Deep size: 103

epoch 0 finished with -0.2780415415763855 loss
epoch 1 finished with -0.2802083492279053 loss
epoch 2 finished with -0.28236088156700134 loss
epoch 3 finished with -0.28828415274620056 loss
epoch 4 finished with -0.2938397526741028 loss
Finished with -0.2938397526741028 loss
>>> Net 3/10
Deep size: 95

epoch 0 finished with -0.23326300084590912 loss
epoch 1 finished with -0.2431890219449997 loss
epoch 2 finished with -0.243187814950943 loss
epoch 3 finished with -0.2427978515625 loss
epoch 4 finished with -0.24886994063854218 loss
Finished with -0.24886994063854218 loss
>>> Net 4/10
Deep size: 101

epoch 0 finished with -0.25699248909950256 loss
epoch 1 fini

In [34]:
nn_model_5 = StackedDNN(10, DNN_Model_5, min_deep=100, max_deep=110)
nn_model_5.fit(data, target, 5, subset=1)
nn_model_5.save()

>>> Net 1/10
Deep size: 103

epoch 0 finished with -0.2240101397037506 loss
epoch 1 finished with -0.2155267745256424 loss
epoch 2 finished with -0.21182452142238617 loss
epoch 3 finished with -0.20135419070720673 loss
epoch 4 finished with -0.21555712819099426 loss
Finished with -0.21555712819099426 loss
>>> Net 2/10
Deep size: 103

epoch 0 finished with -0.2785224914550781 loss
epoch 1 finished with -0.281108021736145 loss
epoch 2 finished with -0.26938337087631226 loss
epoch 3 finished with -0.280425488948822 loss
epoch 4 finished with -0.2702045440673828 loss
Finished with -0.2702045440673828 loss
>>> Net 3/10
Deep size: 108

epoch 0 finished with -0.2590528130531311 loss
epoch 1 finished with -0.26606354117393494 loss
epoch 2 finished with -0.2600416839122772 loss
epoch 3 finished with -0.26438668370246887 loss
epoch 4 finished with -0.2559710443019867 loss
Finished with -0.2559710443019867 loss
>>> Net 4/10
Deep size: 101

epoch 0 finished with -0.251762717962265 loss
epoch 1 fin

In [35]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=12000, task_type='GPU', devices='0', silent=True, subsample=0.9, bootstrap_type='Poisson').fit(data, target)

In [36]:
model.save_model("cat_model")

In [37]:
#res_boost = model.predict(X_test)

In [38]:
"""
best = 0
for i in range(100000):
    a = np.random.rand(7)
    for j in range(7):
        if (np.random.rand() <= 0.06):
            a[j] = 0
    a /= np.sum(a)
    total = pearsonr(a[0]*res + a[1]*res_1 + a[2]*res_2 + a[3]*res_3 + a[4]*res_4 + a[5]*res_5 + a[6]*res_boost, y_test)[0]
    if total > best:
        best = total
        print(a, best, i)
"""

'\nbest = 0\nfor i in range(100000):\n    a = np.random.rand(7)\n    for j in range(7):\n        if (np.random.rand() <= 0.06):\n            a[j] = 0\n    a /= np.sum(a)\n    total = pearsonr(a[0]*res + a[1]*res_1 + a[2]*res_2 + a[3]*res_3 + a[4]*res_4 + a[5]*res_5 + a[6]*res_boost, y_test)[0]\n    if total > best:\n        best = total\n        print(a, best, i)\n'

In [39]:
#[0.001, 0.337, 0.116, 0.01, 0.536] - coefs

In [40]:
#pearsonr(0*res + 0*res_1 + 0*res_2 + 0*res_3 + 1/6*res_4 + 0*res_5, y_test)[0]

In [41]:
#1-0.111-0.43-0.02

In [42]:
"""
a = 0.8
print(pearsonr(res, y_test)[0])
print(pearsonr(res_boost, y_test)[0])
pearsonr(a*res_boost+(1-a)*res, y_test)[0]
"""

'\na = 0.8\nprint(pearsonr(res, y_test)[0])\nprint(pearsonr(res_boost, y_test)[0])\npearsonr(a*res_boost+(1-a)*res, y_test)[0]\n'

In [43]:
"""
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    res = nn_model.predict(test_df[features])
    res_1 = nn_model_1.predict(test_df[features])
    res_2 = nn_model_2.predict(test_df[features])
    res_3 = nn_model_3.predict(test_df[features])
    res_boost = model.predict(test_df[features])
    #[0.001, 0.337, 0.116, 0.01, 0.536]
    test_df['target']  = 0.001*res + 0.337*res_1 + 0.116*res_2 + 0.01*res_3 + 0.536*res_boost
    env.predict(test_df[['row_id','target']])
"""

"\nimport ubiquant\nenv = ubiquant.make_env()   # initialize the environment\niter_test = env.iter_test()    # an iterator which loops over the test set and sample submission\nfor (test_df, sample_prediction_df) in iter_test:\n    res = nn_model.predict(test_df[features])\n    res_1 = nn_model_1.predict(test_df[features])\n    res_2 = nn_model_2.predict(test_df[features])\n    res_3 = nn_model_3.predict(test_df[features])\n    res_boost = model.predict(test_df[features])\n    #[0.001, 0.337, 0.116, 0.01, 0.536]\n    test_df['target']  = 0.001*res + 0.337*res_1 + 0.116*res_2 + 0.01*res_3 + 0.536*res_boost\n    env.predict(test_df[['row_id','target']])\n"